In [1]:
import os
import signal
import time
import csv
import sys
import warnings
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.multiprocessing as mp
import numpy as np
import time
import pprint
from loguru import logger
import wandb
import yaml

from utils import config, logger_tools, other_tools
from dataloaders import data_tools
from dataloaders.build_vocab import Vocab
from optimizers.optim_factory import create_optimizer
from optimizers.scheduler_factory import create_scheduler
from optimizers.loss_factory import get_loss_func

In [2]:
import time
from typing import List, Union

class Config:
    def __init__(self):
        self.config: str = "configs/ae_beat_4english_15_141.yaml"
        self.project: str = "audio2pose"
        self.csv: str = "git3.csv"
        self.trainer: str = "camn"
        self.notes: str = ""

        # Path and save name
        self.is_train: bool = True
        self.root_path: str = ""
        self.out_root_path: str = "/outputs/audio2pose/"
        self.train_data_path: str = "/datasets/trinity/train/"
        self.val_data_path: str = "/datasets/trinity/val/"
        self.test_data_path: str = "/datasets/trinity/test/"
        self.mean_pose_path: str = "/datasets/trinity/train/"
        self.std_pose_path: str = "/datasets/trinity/train/"

        # Pretrained weights
        self.torch_hub_path: str = "../../datasets/checkpoints/"
        self.model_name_last: str = "last.pth"
        self.model_name_best: str = "best.pth"
        self.eval_model: str = "vae"
        self.e_name: Union[str, None] = None
        self.e_path: str = "/datasets/beat/generated_data/self_vae_128.bin"
        self.test_ckpt: str = "/datasets/beat_cache/beat_4english_15_141/last.bin"
        self.variational_encoding: bool = False
        self.vae_length: int = 256

        # Data
        self.dataset: str = "beat"
        self.pose_version: str = "spine_neck_141"
        self.new_cache: bool = True
        self.use_aug: bool = False
        self.disable_filtering: bool = False
        self.clean_first_seconds: int = 0
        self.clean_final_seconds: int = 0

        self.audio_rep: Union[str, None] = None
        self.word_rep: Union[str, None] = None
        self.emo_rep: Union[str, None] = None
        self.sem_rep: Union[str, None] = None
        self.facial_rep: Union[str, None] = None
        self.pose_rep: str = "bvh_rot"
        self.speaker_id: Union[str, None] = None
        self.freeze_wordembed: bool = True
        self.audio_fps: int = 16000
        self.facial_fps: int = 15
        self.pose_fps: int = 15

        self.audio_dims: int = 1
        self.facial_dims: int = 39
        self.pose_dims: int = 123
        self.word_index_num: int = 5793
        self.word_dims: int = 300
        self.speaker_dims: int = 4
        self.emotion_dims: int = 8

        self.audio_norm: bool = False
        self.facial_norm: bool = False
        self.pose_norm: bool = True

        self.pose_length: int = 34
        self.pre_frames: int = 4
        self.stride: int = 10
        self.pre_type: str = "zero"

        self.audio_f: int = 128
        self.facial_f: int = 128
        self.speaker_f: int = 0
        self.word_f: int = 0
        self.emotion_f: int = 0
        self.aud_prob: float = 1.0
        self.pos_prob: float = 1.0
        self.txt_prob: float = 1.0
        self.fac_prob: float = 1.0
        self.multi_length_training: List[float] = [1.0]

        # Model
        self.pretrain: bool = False
        self.model: str = "camn"
        self.g_name: str = "CaMN"
        self.d_name: Union[str, None] = None
        self.dropout_prob: float = 0.3
        self.n_layer: int = 4
        self.hidden_size: int = 300
        self.finger_net: str = "original"

        # Training
        self.epochs: int = 120
        self.grad_norm: int = 0
        self.no_adv_epochs: int = 4
        self.batch_size: int = 128
        self.opt: str = "adam"
        self.lr_base: float = 0.00025
        self.opt_betas: List[float] = [0.5, 0.999]
        self.weight_decay: float = 0.0
        self.lr_min: float = 1e-7
        self.warmup_lr: float = 5e-4
        self.warmup_epochs: int = 0
        self.decay_epochs: int = 9999
        self.decay_rate: float = 0.1
        self.lr_policy: str = "step"
        self.momentum: float = 0.8
        self.nesterov: bool = True
        self.amsgrad: bool = False
        self.d_lr_weight: float = 0.2
        self.rec_weight: float = 500
        self.adv_weight: float = 20.0
        self.fid_weight: float = 0.0
        self.vel_weight: float = 0.0
        self.acc_weight: float = 0.0
        self.kld_weight: float = 0.0
        self.kld_aud_weight: float = 0.0
        self.kld_fac_weight: float = 0.0
        self.ali_weight: float = 0.0
        self.div_reg_weight: float = 0.0
        self.rec_aud_weight: float = 0.0
        self.rec_pos_weight: float = 0.0
        self.rec_fac_weight: float = 0.0
        self.rec_txt_weight: float = 0.0

        # Device
        self.random_seed: int = 2021
        self.deterministic: bool = True
        self.benchmark: bool = True
        self.cudnn_enabled: bool = True
        self.apex: bool = False
        self.gpus: List[int] = [0]
        self.loader_workers: int = 0
        self.ddp: bool = False

        # Logging
        self.log_period: int = 10
        self.test_period: int = 20

        self.name: str = ""

    def update_name(self):
        idc = self.config.rfind('/')
        self.name = self.config[idc+1:-5]
        
        if self.is_train:
            time_local = time.localtime()
            name_expend = f"{time_local[1]:02d}{time_local[2]:02d}_{time_local[3]:02d}{time_local[4]:02d}{time_local[5]:02d}_"
            self.name = name_expend + self.name

    def load_yaml(self):
        with open(self.config, 'r') as file:
            yaml_data = yaml.safe_load(file)
        
        for key, value in yaml_data.items():
            if hasattr(self, key):
                setattr(self, key, value)
            else:
                print(f"Warning: '{key}' is not a valid configuration parameter.")
    
    def print_attributes(self):
        for attr, value in vars(self).items():
            print(f"{attr}: {value}")
            
def create_config() -> Config:
    config = Config()
    config.load_yaml()
    return config

In [3]:
os.environ["MASTER_ADDR"]='localhost'
os.environ["MASTER_PORT"]='2223'
args = create_config()
args.update_name()

@logger.catch
def main_worker(rank, world_size, args):
    if not sys.warnoptions:
        warnings.simplefilter("ignore")
    dist.init_process_group("nccl", rank=rank, world_size=world_size)
        
    logger_tools.set_args_and_logger(args, rank)
    other_tools.set_random_seed(args)
    other_tools.print_exp_info(args)

main_worker(0, 1, args)

test_data = __import__(f"dataloaders.{args.dataset}", fromlist=["something"]).CustomDataset(args, "test")



2024-09-12 06:17:37.664 | ERROR    | __main__:<module>:16 - An error has been caught in function '<module>', process 'MainProcess' (2076401), thread 'MainThread' (127008750913344):
Traceback (most recent call last):

  File "/opt/miniconda/envs/3d/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
           │         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
           │         └ <code object <module> at 0x7383872c7b30, file "/opt/miniconda/envs/3d/lib/python3.8/site-packages/ipykernel_launcher.py", lin...
           └ <function _run_code at 0x7383872d4c10>
  File "/opt/miniconda/envs/3d/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, run_globals)
         │     └ {'__name__': '__main__', '__doc__': 'Entry point for launching an IPython kernel.\n\nThis is separate from the ipykernel pack...
         └ <code object <module> 

: 

In [7]:
test_data[0]['pose'].shape

torch.Size([450, 225])